Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [12]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [13]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [29]:
batch_size = 128
beta_reg = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + beta_reg*(tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.626564
Minibatch accuracy: 10.9%
Validation accuracy: 15.9%
Minibatch loss at step 500: 0.959012
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.670784
Minibatch accuracy: 86.7%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.793549
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.801387
Minibatch accuracy: 78.1%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 0.759514
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.557222
Minibatch accuracy: 86.7%
Validation accuracy: 81.8%
Test accuracy: 89.0%


In [25]:
batch_size = 128
n_relus = 1024
beta_reg = 0.01
graph2 = tf.Graph()
with graph2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_relus]))
  biases_i = tf.Variable(tf.zeros([n_relus]))
  weights_o = tf.Variable(
    tf.truncated_normal([n_relus, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_i = tf.matmul(tf_train_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_o))
  loss = loss + beta_reg*(tf.nn.l2_loss(weights_i)+tf.nn.l2_loss(weights_o))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_valid_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  valid_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_test_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  test_prediction = tf.nn.softmax(logits_o)
  

In [26]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3468.571533
Minibatch accuracy: 10.2%
Validation accuracy: 32.4%
Minibatch loss at step 500: 21.271023
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.799178
Minibatch accuracy: 85.9%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 0.778373
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 0.784037
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.722355
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 0.577489
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
batch_size = 128
beta_reg = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss + beta_reg*(tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [37]:
num_steps = 3001
train_dataset_limited = train_dataset[:400,:]
train_labels_limited = train_labels[:400,:]
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_limited.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_limited[offset:(offset + batch_size), :]
    batch_labels = train_labels_limited[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.663658
Minibatch accuracy: 4.7%
Validation accuracy: 7.1%
Minibatch loss at step 500: 0.420178
Minibatch accuracy: 99.2%
Validation accuracy: 74.6%
Minibatch loss at step 1000: 0.224929
Minibatch accuracy: 99.2%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.272729
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 2000: 0.249430
Minibatch accuracy: 99.2%
Validation accuracy: 75.2%
Minibatch loss at step 2500: 0.229694
Minibatch accuracy: 99.2%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 0.243492
Minibatch accuracy: 99.2%
Validation accuracy: 76.8%
Test accuracy: 84.2%


In [38]:
batch_size = 128
n_relus = 1024
beta_reg = 0.01
graph2 = tf.Graph()
with graph2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_relus]))
  biases_i = tf.Variable(tf.zeros([n_relus]))
  weights_o = tf.Variable(
    tf.truncated_normal([n_relus, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_i = tf.matmul(tf_train_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_o))
  loss = loss + beta_reg*(tf.nn.l2_loss(weights_i)+tf.nn.l2_loss(weights_o))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_valid_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  valid_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_test_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  test_prediction = tf.nn.softmax(logits_o)
  

In [39]:
num_steps = 3001
train_dataset_limited = train_dataset[:400,:]
train_labels_limited = train_labels[:400,:]
with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_limited.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_limited[offset:(offset + batch_size), :]
    batch_labels = train_labels_limited[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3565.957520
Minibatch accuracy: 10.9%
Validation accuracy: 40.8%
Minibatch loss at step 500: 21.004984
Minibatch accuracy: 99.2%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 0.423628
Minibatch accuracy: 99.2%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 0.290059
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.267077
Minibatch accuracy: 99.2%
Validation accuracy: 76.0%
Minibatch loss at step 2500: 0.249911
Minibatch accuracy: 99.2%
Validation accuracy: 77.4%
Minibatch loss at step 3000: 0.255694
Minibatch accuracy: 99.2%
Validation accuracy: 77.5%
Test accuracy: 84.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [47]:
batch_size = 128
n_relus = 1024
beta_reg = 0.01
graph2 = tf.Graph()
with graph2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_i = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_relus]))
  biases_i = tf.Variable(tf.zeros([n_relus]))
  weights_o = tf.Variable(
    tf.truncated_normal([n_relus, num_labels]))
  biases_o = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits_i = tf.matmul(tf_train_dataset, weights_i) + biases_i
  layer_r = tf.nn.dropout(tf.nn.relu(logits_i),keep_prob=0.5,seed=0)
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_o))
  loss = loss + beta_reg*(tf.nn.l2_loss(weights_i)+tf.nn.l2_loss(weights_o))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_valid_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  valid_prediction = tf.nn.softmax(logits_o)

  logits_i = tf.matmul(tf_test_dataset, weights_i) + biases_i
  layer_r = tf.nn.relu(logits_i)  
  logits_o = tf.matmul(layer_r, weights_o) + biases_o 
  test_prediction = tf.nn.softmax(logits_o)
  

In [48]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3622.123047
Minibatch accuracy: 7.8%
Validation accuracy: 33.1%
Minibatch loss at step 500: 21.493111
Minibatch accuracy: 84.4%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 0.905042
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 0.866284
Minibatch accuracy: 82.0%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 0.886269
Minibatch accuracy: 80.5%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.780598
Minibatch accuracy: 85.9%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 0.633073
Minibatch accuracy: 88.3%
Validation accuracy: 83.0%
Test accuracy: 90.0%


In [45]:
num_steps = 3001
train_dataset_limited = train_dataset[:400,:]
train_labels_limited = train_labels[:400,:]
with tf.Session(graph=graph2) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_limited.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_limited[offset:(offset + batch_size), :]
    batch_labels = train_labels_limited[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3625.794189
Minibatch accuracy: 12.5%
Validation accuracy: 29.7%
Minibatch loss at step 500: 21.117905
Minibatch accuracy: 99.2%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 0.447986
Minibatch accuracy: 99.2%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 0.313383
Minibatch accuracy: 99.2%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 0.296700
Minibatch accuracy: 98.4%
Validation accuracy: 76.1%
Minibatch loss at step 2500: 0.268326
Minibatch accuracy: 99.2%
Validation accuracy: 77.4%
Minibatch loss at step 3000: 0.271506
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
